**Basit seviye Cirq ile devre tasarımı**

In [1]:
# Google Colab ortamında çalışıyorsanız
!pip install cirq --quiet
!pip install scikit-learn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.5/597.5 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.1/425.1 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 101.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import cirq
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize


In [3]:
# Veri kümesini yükle
data = load_breast_cancer()
X = data.data
y = data.target

# Yalnızca ilk 4 özelliği kullanacağız (kuantum devresinde çok fazla kübit zor olur)
X = X[:, :4]

# Veriyi ölçeklendir
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Eğitim ve test bölmesi
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [4]:
#Kuantum devresi tanımı
#Özellikleri kuantuma gömme

def feature_map(qubits, x):
    for i, qubit in enumerate(qubits):
        cirq.H(qubit)  # Süperpozisyon
        cirq.rx(x[i])(qubit)  # Özellikleri RX rotasyonu ile kodla


In [5]:
#Parametrik ansatz
def ansatz(qubits, params):
    idx = 0
    for qubit in qubits:
        cirq.ry(params[idx])(qubit)
        idx += 1
    for i in range(len(qubits) - 1):
        cirq.CNOT(qubits[i], qubits[i+1])


In [6]:
#Devreyi çalıştırmak ve ölçüm yapmak
def create_circuit(x, params):
    qubits = cirq.GridQubit.rect(1, len(x))
    circuit = cirq.Circuit()
    feature_map(qubits, x)
    ansatz(qubits, params)
    circuit.append(cirq.measure(*qubits, key='m'))
    return circuit, qubits


In [7]:
#Kayıp fonksiyonu
#Klasik optimizasyon için bir kayıp fonksiyonu tanımlayacağız (ör. çapraz entropi yerine MSE gibi).
simulator = cirq.Simulator()

def predict(x, params):
    circuit, qubits = create_circuit(x, params)
    result = simulator.run(circuit, repetitions=100)
    measurements = result.measurements['m']
    # Örneğin ilk kübitin ölçümünü ortalama olarak al
    mean = np.mean(measurements[:, 0])
    return mean

def cost(params, X, y):
    preds = []
    for i in range(len(X)):
        pred = predict(X[i], params)
        preds.append(pred)
    preds = np.array(preds)
    loss = np.mean((preds - y)**2)
    return loss


In [8]:
# Eğitimi başlat
# Başlangıç parametreleri
init_params = np.random.uniform(0, 2*np.pi, size=4)

result = minimize(cost, init_params, args=(X_train, y_train), method='COBYLA', options={'maxiter': 50})

print("Optimize edilmiş parametreler:", result.x)


Optimize edilmiş parametreler: [6.06750947 4.40707186 3.13341579 0.5639194 ]


In [10]:
#Test et
def evaluate(X, y, params):
    preds = []
    for i in range(len(X)):
        pred = predict(X[i], params)
        preds.append(int(pred > 0.5))
    accuracy = np.mean(np.array(preds) == y)
    return accuracy

acc_train = evaluate(X_train, y_train, result.x)
acc_test = evaluate(X_test, y_test, result.x)

print(f"Eğitim doğruluğu: {acc_train:.2f}")
print(f"Test doğruluğu: {acc_test:.2f}")


Eğitim doğruluğu: 0.37
Test doğruluğu: 0.38


Neden böyle düşük çıktı?

✅ Özellik sayısı çok az.

Breast Cancer veri seti aslında 30 özellik içeriyor.

Biz sadece 4 özellik kullandık ve en iyi olanları değil, ilk 4 taneyi seçtik. Özellik mühendisliği yapmadık.

✅ Ansatz çok basit.

Sadece bir katman parametrik Ry ve birkaç CNOT ile derinliği çok sığ bir devre kurduk.

QNN’in model kapasitesi çok düşük.

✅ Ölçüm ve hedef arasındaki ilişki zayıf.

Sadece bir kübitin ortalamasını ölçüp onu 0/1 ile karşılaştırıyoruz. Bu, “label”larla iyi örtüşmeyebilir.

Ölçüm sonuçları 0–1 arasında ortalama bir değer veriyor ama biz bunu lineer olarak 0/1 sınıfıyla kıyaslıyoruz.

✅ Optimizasyon yöntemi zayıf.

COBYLA kullanıldı, daha iyi optimizasyon yöntemleri (Adam, SPSA) denenebilir.

Ayrıca tekrar sayısı (100) düşük ve gürültülü.

**Cirq ile daha iyi bir versiyon**

Cirq ile Geliştirilmiş QNN Uygulaması

In [11]:
# Google Colab için:
!pip install cirq --quiet
!pip install scikit-learn --quiet


In [12]:
import numpy as np
import cirq
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.optimize import minimize


In [13]:
# Veri kümesini yükle
data = load_breast_cancer()
X = data.data
y = data.target

# En iyi 4 özelliği seç
selector = SelectKBest(f_classif, k=4)
X_new = selector.fit_transform(X, y)

# Veriyi ölçekle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# Eğitim ve test seti
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [14]:
def feature_map(qubits, x):
    for i, qubit in enumerate(qubits):
        cirq.H(qubit)
        cirq.rx(x[i])(qubit)
        cirq.rz(x[i])(qubit)


In [15]:
def ansatz(qubits, params, layers=2):
    idx = 0
    for l in range(layers):
        for qubit in qubits:
            cirq.ry(params[idx])(qubit)
            idx += 1
        for i in range(len(qubits) - 1):
            cirq.CNOT(qubits[i], qubits[i+1])


In [16]:
def create_circuit(x, params):
    qubits = cirq.GridQubit.rect(1, len(x))
    circuit = cirq.Circuit()
    feature_map(qubits, x)
    ansatz(qubits, params)
    circuit.append(cirq.measure(*qubits, key='m'))
    return circuit, qubits


In [17]:
#Kayıp Fonksiyonu
#Ölçüm stratejisi: tüm kübitlerin ortalaması
simulator = cirq.Simulator()

def predict(x, params):
    circuit, qubits = create_circuit(x, params)
    result = simulator.run(circuit, repetitions=200)
    measurements = result.measurements['m']
    mean = np.mean(measurements)
    return mean

def cost(params, X, y):
    preds = []
    for i in range(len(X)):
        pred = predict(X[i], params)
        preds.append(pred)
    preds = np.array(preds)
    # normalize hedef
    y_norm = y
    loss = np.mean((preds - y_norm)**2)
    return loss


In [18]:
n_params = 4 * 2  # 4 kübit x 2 katman
init_params = np.random.uniform(0, 2*np.pi, size=n_params)

result = minimize(cost, init_params, args=(X_train, y_train), method='COBYLA', options={'maxiter': 50})

print("Optimize edilmiş parametreler:", result.x)


Optimize edilmiş parametreler: [2.23570744 4.25354824 3.18701188 4.73968316 4.95323827 1.70594543
 4.23274112 3.64391747]


In [21]:
def evaluate(X, y, params):
    preds = []
    for i in range(len(X)):
        pred = predict(X[i], params)
        preds.append(int(pred > 0.5))
    accuracy = np.mean(np.array(preds) == y)
    return accuracy

acc_train = evaluate(X_train, y_train, result.x)
acc_test = evaluate(X_test, y_test, result.x)

print(f"Eğitim doğruluğu: {acc_train:.2f}")
print(f"Test doğruluğu: {acc_test:.2f}")


Eğitim doğruluğu: 0.37
Test doğruluğu: 0.38


Cirq üzerinde kuantum devre ile optimize ettiğimiz model klasik yöntemlere göre çok düşük doğruluk veriyor.

Bu, Cirq’ün QNN için uygun bir “yüksek seviyeli” kütüphane olmamasından kaynaklanıyor.

Ayrıca, klasik optimizasyonla parametrik kuantum devreyi eğitmek çok zor bir problem (vanishing gradients ve gürültü nedeniyle).



**PennyLane ile QNN Uygulaması**


In [22]:
!pip install pennylane --quiet
!pip install scikit-learn --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 127.6 MB/s eta 0:00:00


In [23]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif


In [24]:
# Veri kümesini yükle
data = load_breast_cancer()
X = data.data
y = data.target

# En iyi 4 özelliği seç
selector = SelectKBest(f_classif, k=4)
X_new = selector.fit_transform(X, y)

# Veriyi ölçekle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# Eğitim ve test bölmesi
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [25]:
#PennyLane cihazı
#4 kübitlik devre tanımlıyoruz.

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)


In [26]:
def qnn_circuit(x, weights):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RX(x[i], wires=i)
        qml.RZ(x[i], wires=i)

    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))


In [27]:
weight_shapes = {"weights": (3, n_qubits, 3)}  # 3 katmanlık entangling layer

qnode = qml.QNode(qnn_circuit, dev, interface="autograd")


In [28]:
#Kayıp Fonksiyonu ve Optimizer
def cost(weights, X, y):
    preds = [qnode(x, weights) for x in X]
    preds = np.array(preds)
    # PauliZ expectation -1 ~ 0, +1 ~ 1 --> normalize
    preds = (preds + 1) / 2
    loss = np.mean((preds - y) ** 2)
    return loss


In [29]:
weights = 0.01 * np.random.randn(3, n_qubits, 3, requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 20

for epoch in range(epochs):
    weights = opt.step(lambda w: cost(w, X_train, y_train), weights)
    train_loss = cost(weights, X_train, y_train)
    print(f"Epoch {epoch+1}: Loss = {train_loss:.4f}")


Epoch 1: Loss = 0.2208
Epoch 2: Loss = 0.1881
Epoch 3: Loss = 0.1623
Epoch 4: Loss = 0.1424
Epoch 5: Loss = 0.1217
Epoch 6: Loss = 0.1015
Epoch 7: Loss = 0.0877
Epoch 8: Loss = 0.0822
Epoch 9: Loss = 0.0805
Epoch 10: Loss = 0.0787
Epoch 11: Loss = 0.0755
Epoch 12: Loss = 0.0719
Epoch 13: Loss = 0.0693
Epoch 14: Loss = 0.0675
Epoch 15: Loss = 0.0656
Epoch 16: Loss = 0.0635
Epoch 17: Loss = 0.0611
Epoch 18: Loss = 0.0587
Epoch 19: Loss = 0.0568
Epoch 20: Loss = 0.0558


In [30]:
def evaluate(X, y, weights):
    preds = [qnode(x, weights) for x in X]
    preds = np.array(preds)
    preds = (preds + 1) / 2
    preds = (preds > 0.5).astype(int)
    accuracy = np.mean(preds == y)
    return accuracy

acc_train = evaluate(X_train, y_train, weights)
acc_test = evaluate(X_test, y_test, weights)

print(f"Eğitim doğruluğu: {acc_train:.2f}")
print(f"Test doğruluğu: {acc_test:.2f}")


Eğitim doğruluğu: 0.94
Test doğruluğu: 0.94


| Yöntem                        | Eğitim Doğruluğu | Test Doğruluğu |
| ----------------------------- | ---------------- | -------------- |
| **Cirq (elle yazılmış QNN)**  | \~0.37           | \~0.38         |
| **PennyLane (QNN framework)** | **0.94**         | **0.94**       |

### 🔍 Neden PennyLane daha iyi?

✅ Yüksek seviyeli `StronglyEntanglingLayers` ansatz kullandı (çok daha güçlü).

✅ Optimize edilmiş bir eğitim döngüsü ve `AdamOptimizer`.

✅ Daha iyi otomatik türev (autograd) desteği.

✅ Daha stabil optimizasyon ve iyi ölçüm stratejisi.


**TFQ ile QNN Uygulaması**


In [1]:
!pip install tensorflow==2.15 --quiet
!pip install tensorflow-quantum --quiet
!pip install scikit-learn --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.15.0 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.15.0 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but

In [1]:
pip uninstall -y tensorflow tensorflow-io tensorflow-text tensorflow-decision-forests tf-keras
pip install tensorflow==2.15 tensorflow-quantum scikit-learn protobuf==4.25.3 --quiet


SyntaxError: invalid syntax (ipython-input-1-3272092742.py, line 1)

In [2]:
pip install tensorflow-quantum scikit-learn --quiet


In [3]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif


In [4]:
# Veri kümesini yükle
data = load_breast_cancer()
X = data.data
y = data.target

# En iyi 4 özelliği seç
selector = SelectKBest(f_classif, k=4)
X_new = selector.fit_transform(X, y)

# Veriyi ölçekle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# Eğitim ve test bölmesi
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [5]:
# Kuantum devresi
# Qubit ve semboller

n_qubits = 4
qubits = [cirq.GridQubit(0, i) for i in range(n_qubits)]

# Özellikleri sembollere bağla
data_symbols = [sympy.Symbol(f'x{i}') for i in range(n_qubits)]
weights = sympy.symbols('theta(0:12)')  # 3 katman * 4 kübit


In [6]:
# Devre tanımı
def create_qnn_circuit():
    circuit = cirq.Circuit()
    # Özellikleri encode et
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(data_symbols[i])(qubit))
        circuit.append(cirq.rz(data_symbols[i])(qubit))

    # Parametrik katmanlar
    k = 0
    for _ in range(3):  # 3 katman
        for i, qubit in enumerate(qubits):
            circuit.append(cirq.ry(weights[k])(qubit))
            k += 1
        for i in range(n_qubits - 1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
    return circuit


In [7]:
# Devreleri ve veri setini hazırla
# Eğitim ve test için devreleri oluştur
qnn_circuit = create_qnn_circuit()
x_train_circuits = [cirq.Circuit() for _ in range(len(X_train))]
x_test_circuits = [cirq.Circuit() for _ in range(len(X_test))]

for i, x in enumerate(X_train):
    x_train_circuits[i] = cirq.resolve_parameters(qnn_circuit, dict(zip(data_symbols, x)))
for i, x in enumerate(X_test):
    x_test_circuits[i] = cirq.resolve_parameters(qnn_circuit, dict(zip(data_symbols, x)))

# TensorFlow veri kümeleri
x_train_tfq = tfq.convert_to_tensor(x_train_circuits)
x_test_tfq = tfq.convert_to_tensor(x_test_circuits)

y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)


In [8]:
#TFQ QNN Modeli
#Katman tanımı

readout = cirq.Z(qubits[0])

qnn_layer = tfq.layers.PQC(qnn_circuit, readout)


In [9]:
model = tf.keras.Sequential([
    qnn_layer,
    tf.keras.layers.Activation('sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [10]:
history = model.fit(x_train_tfq, y_train_tf,
                    batch_size=16,
                    epochs=20,
                    validation_data=(x_test_tfq, y_test_tf))


Epoch 1/20
29/29 [==============================] - 3s 40ms/step - loss: 0.6023 - accuracy: 0.7538 - val_loss: 0.5358 - val_accuracy: 0.8246
Epoch 2/20
29/29 [==============================] - 1s 30ms/step - loss: 0.5246 - accuracy: 0.8703 - val_loss: 0.4997 - val_accuracy: 0.9035
Epoch 3/20
29/29 [==============================] - 1s 31ms/step - loss: 0.5138 - accuracy: 0.8747 - val_loss: 0.4903 - val_accuracy: 0.9123
Epoch 4/20
29/29 [==============================] - 1s 29ms/step - loss: 0.5124 - accuracy: 0.8835 - val_loss: 0.4934 - val_accuracy: 0.9123
Epoch 5/20
29/29 [==============================] - 1s 38ms/step - loss: 0.5065 - accuracy: 0.8879 - val_loss: 0.5010 - val_accuracy: 0.8596
Epoch 6/20
29/29 [==============================] - 1s 40ms/step - loss: 0.5091 - accuracy: 0.8813 - val_loss: 0.4969 - val_accuracy: 0.8947
Epoch 7/20
29/29 [==============================] - 1s 32ms/step - loss: 0.5093 - accuracy: 0.8835 - val_loss: 0.4959 - val_accuracy: 0.8860
Epoch 8/20
29

In [11]:
train_acc = history.history['accuracy'][-1]
test_acc = history.history['val_accuracy'][-1]

print(f"Eğitim doğruluğu: {train_acc:.2f}")
print(f"Test doğruluğu: {test_acc:.2f}")


Eğitim doğruluğu: 0.89
Test doğruluğu: 0.86


| Yöntem              | Eğitim Doğruluğu | Test Doğruluğu |
| ------------------- | ---------------- | -------------- |
| **Cirq (elle QNN)** | \~0.37           | \~0.38         |
| **PennyLane (QNN)** | **0.94**         | **0.94**       |
| **TFQ (QNN)**       | **0.89**         | **0.86**       |


✅ Cirq (elle):

Elle yazılmış devre ve klasik optimizer kullanmak yetersiz kaldı.

QNN framework’ü olmadığı için başarım kötüydü.

✅ PennyLane:

QNN için optimize edilmiş katmanlar (StronglyEntanglingLayers), iyi optimizer (Adam), otomatik türev desteği sayesinde çok yüksek doğruluk verdi.

Genel olarak en iyi sonucu verdi.

✅ TFQ:

Klasik TensorFlow ile çok iyi entegre.

Sonuçlar iyi, ama PennyLane kadar yüksek değil.

Daha fazla epoch veya hiperparametre ayarı ile biraz daha iyileştirilebilir.*italik metin*


**Qiskit ile QNN Uygulaması**


In [ ]:
pip install qiskit --quiet
pip install qiskit-machine-learning --quiet
pip install qiskit-algorithms --quiet
pip install scikit-learn --quiet


In [49]:
pip install pylatexenc --quiet


In [50]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA


In [51]:
# Veri kümesini yükle
data = load_breast_cancer()
X = data.data
y = data.target

# En iyi 4 özelliği seç
selector = SelectKBest(f_classif, k=4)
X_new = selector.fit_transform(X, y)

# Ölçekle
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# Eğitim / test bölmesi
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [52]:
# Kuantum devresi
# Parametreler

num_qubits = 4
sym_x = ParameterVector('x', num_qubits)
sym_w = ParameterVector('w', num_qubits)


In [54]:
#Devre

qc = QuantumCircuit(num_qubits)

# Feature map
for i in range(num_qubits):
    qc.h(i)
    qc.ry(sym_x[i], i)
    qc.rz(sym_x[i], i)

# Ansatz
for i in range(num_qubits):
    qc.ry(sym_w[i], i)

# Ölçüm ekleme! (Burada yok)
# qc.draw('mpl')  # opsiyonel


In [70]:
#QNN
estimator = Estimator()

qnn = EstimatorQNN(
    circuit=qc,
    input_params=sym_x,
    weight_params=sym_w
)


/tmp/ipython-input-70-3458938400.py:3: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [ ]:
#Sınıflayıcı
#Optimizer
optimizer = COBYLA(maxiter=50)


In [57]:
#Model
classifier = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    initial_point=np.random.rand(num_qubits)
)


In [58]:
classifier.fit(X_train, y_train)


In [59]:
train_acc = classifier.score(X_train, y_train)
test_acc = classifier.score(X_test, y_test)

print(f"Eğitim doğruluğu: {train_acc:.2f}")
print(f"Test doğruluğu: {test_acc:.2f}")


Eğitim doğruluğu: 0.62
Test doğruluğu: 0.61


Neden düşük kaldı?

Qiskit’te QNN için kullandığımız yapı:

Çok basit bir devre (sadece 4 qubit, 1 katman Ry ansatz)

Özellik haritası (feature map) çok zayıf (sadece Ry ve Rz)

COBYLA optimizer fazla iyi ölçeklenmez, lokal minimumda kalabilir

Epoch ve maxiter küçük (biz 50 kullandık; 300–500 daha iyi sonuç verir)

🔷 Ne yapılabilir?

✅ Daha güçlü bir ansatz eklenebilir:

Entanglement ekle (örneğin: CNOT zinciri)

Daha fazla katman (depth)

Daha fazla maxiter

✅ Daha iyi optimizer:

SPSA (QNN’lerde genelde daha iyi sonuç verir)

✅ Daha iyi feature map:

ZZFeatureMap (Qiskit’in klasik örneklerinde kullanılır)

**Qiskit QNN (Geliştirilmiş)**

In [60]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import SPSA


In [61]:
data = load_breast_cancer()
X = data.data
y = data.target

selector = SelectKBest(f_classif, k=4)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (455, 4), y_train shape: (455,)


In [62]:
num_qubits = 4
sym_x = ParameterVector('x', num_qubits)
sym_w = ParameterVector('w', num_qubits * 3)  # daha fazla parametre


In [64]:
# Geliştirilmiş devre
# devremize:
# Özellik haritası: Rx-Rz + daha fazla derinlik
# Ansatz: Ry + CNOT zinciri + 3 katman ekliyoruz


qc = QuantumCircuit(num_qubits)

# Feature map
for i in range(num_qubits):
    qc.rx(sym_x[i], i)
    qc.rz(sym_x[i], i)

# 3 katman ansatz
k = 0
for _ in range(3):  # derinlik
    for i in range(num_qubits):
        qc.ry(sym_w[k], i)
        k += 1
    for i in range(num_qubits - 1):
        qc.cx(i, i+1)

# qc.draw('mpl')  # opsiyonel


In [65]:
estimator = Estimator()

qnn = EstimatorQNN(
    circuit=qc,
    input_params=sym_x,
    weight_params=sym_w
)


/tmp/ipython-input-65-4087120848.py:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/tmp/ipython-input-65-4087120848.py:3: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [66]:
# Daha iyi optimizer
# SPSA kuantum devrelerde en çok önerilen optimizer’dır:
optimizer = SPSA(maxiter=300)


In [67]:
classifier = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    initial_point=np.random.rand(len(sym_w))
)


In [68]:
classifier.fit(X_train, y_train)


In [69]:
train_acc = classifier.score(X_train, y_train)
test_acc = classifier.score(X_test, y_test)

print(f"Eğitim doğruluğu: {train_acc:.2f}")
print(f"Test doğruluğu: {test_acc:.2f}")


Eğitim doğruluğu: 0.63
Test doğruluğu: 0.62


| Yöntem              | Eğitim Doğruluğu | Test Doğruluğu |
| ------------------- | ---------------- | -------------- |
| **Cirq (elle QNN)** | \~0.37           | \~0.38         |
| **PennyLane (QNN)** | **0.94**         | **0.94**       |
| **TFQ (QNN)**       | **0.89**         | **0.86**       |
| **Qiskit (QNN)**    |**0.63**         | **0.62**       |